# Secrets of Laughter: When, What, and Why Audiences Laugh in Talk show 


## Step 0：环境准备

In [ ]:
# 这是一个环境准备的代码，确保以下内容能够完整运行

import os, sys, platform, subprocess

# === 1️⃣ 防止 OMP / MKL 冲突 ===
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"   
os.environ["OMP_NUM_THREADS"] = "1"         
os.environ["MKL_NUM_THREADS"] = "1"

# === 2️⃣ 禁用 matplotlib GUI 模式，防止 VSCode 渲染崩溃 ===
import matplotlib
matplotlib.use("Agg")

# === 3️⃣ 自动设置国内 pip 源（防 ProxyError） ===
try:
    subprocess.run(
        [sys.executable, "-m", "pip", "config", "set", "global.index-url",
         "https://pypi.tuna.tsinghua.edu.cn/simple"],
        check=False,
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )
except Exception:
    pass

# === 4️⃣ 检查 Python 版本 ===
ver = sys.version_info
print(f"🐍 当前 Python 版本: {ver.major}.{ver.minor}.{ver.micro}")
if ver.major == 3 and ver.minor >= 13:
    print("⚠️ 检测到 Python 3.13，不完全兼容 Whisper / SentenceTransformers。建议切换到 Python 3.10 或 3.11 环境运行。")

# === 5️⃣ 检查 FFmpeg 可用性 ===
import shutil
if shutil.which("ffmpeg"):
    print("🎬 FFmpeg 已检测到。")
else:
    print("⚠️ 未检测到 FFmpeg，请安装并加入 PATH（或放到 C:\\ffmpeg\\bin）。")

# === 6️⃣ 打印系统环境信息 ===
print(f"🧠 系统: {platform.system()} {platform.release()} ({platform.machine()})")
print(f"📂 工作目录: {os.getcwd()}")
print(f"✅ 环境变量防护加载完成！可安全执行 Step 0 ~ Step 4")


🐍 当前 Python 版本: 3.13.5
⚠️ 检测到 Python 3.13，不完全兼容 Whisper / SentenceTransformers。建议切换到 Python 3.10 或 3.11 环境运行。
🎬 FFmpeg 已检测到。
🧠 系统: Windows 11 (AMD64)
📂 工作目录: c:\Users\koto\Desktop\SemesterA\5507作业
✅ 环境变量防护加载完成！可安全执行 Step 0 ~ Step 4


In [3]:
import os, sys, subprocess, shutil, platform
import matplotlib
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
matplotlib.use('Agg')
try:
    subprocess.run([sys.executable, '-m', 'pip', 'config', 'set', 'global.index-url', 'https://pypi.tuna.tsinghua.edu.cn/simple'], check=False)
except Exception:
    pass
def ensure(package):
    try:
        __import__(package)
        print(f'✅ {package} 已安装')
    except ImportError:
        print(f'📦 安装 {package} ...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-U', package])
pkgs = ['requests','pandas','matplotlib','seaborn','jieba','networkx','snownlp','tqdm','whisper','sentence-transformers','scipy']
for p in pkgs:
    ensure(p)
print('\n🐍 Python:', sys.version)
print('🧠 系统:', platform.system(), platform.release(), platform.machine())
print('📂 工作目录:', os.getcwd())
print('🎬 FFmpeg:', '已检测到' if shutil.which('ffmpeg') else '未检测到（请安装并加入 PATH）')


✅ requests 已安装
✅ pandas 已安装
✅ matplotlib 已安装
✅ seaborn 已安装
✅ jieba 已安装
✅ networkx 已安装
✅ snownlp 已安装
✅ tqdm 已安装
✅ whisper 已安装
📦 安装 sentence-transformers ...
✅ scipy 已安装

🐍 Python: 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 16:37:03) [MSC v.1929 64 bit (AMD64)]
🧠 系统: Windows 11 AMD64
📂 工作目录: c:\Users\koto\Desktop\SemesterA\5507作业
🎬 FFmpeg: 已检测到


## Step 1：腾讯视频弹幕

In [ ]:
# 这是爬取腾讯视频弹幕的代码
import requests
import pandas as pd
import time
import os
import subprocess
import json
from datetime import datetime
from pathlib import Path

# ================== 配置区域 ==================
cover_id = "mzc00200li7mfpr"  # 节目封面ID
vid = "v41013vxrfa"           # 视频VID
video_path = r"C:\Users\koto\Desktop\talkshow01x.mkv"  # 本地视频路径，可自动读取时长
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/140.0.0.0 Safari/537.36"
    )
}
step = 30000  # 每段时长（毫秒，30秒）
max_retry = 3
# ============================================


# 🎬 自动检测视频总时长
def get_video_duration(video_path: str) -> int:
    try:
        cmd = [
            "ffprobe", "-v", "error", "-show_entries",
            "format=duration", "-of", "default=noprint_wrappers=1:nokey=1",
            video_path
        ]
        result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        duration = float(result.stdout.strip())
        print(f"🎬 自动检测到视频时长：{duration:.2f} 秒")
        return int(duration * 1000)  # 转为毫秒
    except Exception as e:
        print(f"⚠️ 无法获取视频时长: {e}")
        return 0


# 🌐 自动检测接口模板
def detect_url_template(cover_id, vid):
    url_patterns = [
        "https://dm.video.qq.com/barrage/segment/{vid}/t/v1/{start}/{end}",     # 普通接口
        "https://dm.video.qq.com/barrage/segment/{cover}/{vid}/t/v1/{start}/{end}",  # 含cover接口
        "https://dm.video.qq.com/barrage/base/{vid}?barrage_segment={start}",   # 旧版接口
    ]
    print("🔍 正在测试可用接口 ...")

    for pattern in url_patterns:
        test_url = pattern.format(cover=cover_id, vid=vid, start=0, end=30000)
        try:
            r = requests.get(test_url, headers=headers, timeout=8)
            if r.status_code == 200 and "barrage" in r.text:
                print(f"✅ 找到可用接口: {pattern}")
                return pattern
            else:
                print(f"❌ 无效接口: {pattern} (status {r.status_code})")
        except Exception as e:
            print(f"⚠️ 测试失败: {pattern} → {e}")
    print("🛑 未找到可用的弹幕接口。")
    return None


# 💬 抓取单个时间段弹幕
def fetch_segment(url, retries=3):
    for attempt in range(retries):
        try:
            resp = requests.get(url, headers=headers, timeout=10)
            resp.raise_for_status()
            return resp.json()
        except Exception as e:
            print(f"⚠️ 请求失败（第 {attempt + 1} 次）: {e}")
            time.sleep(1)
    return None


# 我要开始抓弹幕啦！
def crawl_danmu(cover_id, vid, video_path):
    # 1. 获取时长（毫秒）
    duration_ms = get_video_duration(video_path)
    if duration_ms == 0:
        print("⚠️ 使用默认时长 10 分钟。")
        duration_ms = 600_000

    # 2. 检测接口模板
    template = detect_url_template(cover_id, vid)
    if not template:
        return

    # 3. 抓取弹幕
    print(f"\n🚀 开始抓取弹幕 (step={step} ms) ...")
    all_danmu = []
    rounds = duration_ms // step + 1

    for i in range(rounds):
        start = i * step
        end = start + step
        url = template.format(cover=cover_id, vid=vid, start=start, end=end)

        data = fetch_segment(url)
        if not data:
            print(f"🛑 第 {i} 段抓取失败，跳过。")
            continue

        barrage_list = data.get("barrage_list") or data.get("barrage_list_v2") or []
        if not barrage_list:
            print(f"🛑 第 {i} 段没有弹幕，结束抓取。")
            break

        print(f"✅ 抓取第 {i} 段，共 {len(barrage_list)} 条")
        for item in barrage_list:
            offset_ms = item.get("time_offset", item.get("start", 0))
            offset_sec = round(float(offset_ms) / 1000, 2)
            offset_mmss = f"{int(offset_sec // 60):02d}:{int(offset_sec % 60):02d}"

            send_ts = item.get("create_time", "")
            if send_ts:
                try:
                    send_time_local = datetime.fromtimestamp(int(send_ts)).strftime("%Y-%m-%d %H:%M:%S")
                except:
                    send_time_local = ""
            else:
                send_time_local = ""

            all_danmu.append({
                "video_id": vid,
                "offset_time_sec": offset_sec,
                "offset_time_mmss": offset_mmss,
                "content": item.get("content", ""),
                "send_time": send_time_local,
                "user_name": (
                    item.get("user", {}).get("nick", "")
                    if isinstance(item.get("user"), dict)
                    else item.get("nick", "")
                ),
                "color": item.get("color", ""),
                "mode": item.get("mode", ""),
            })
        time.sleep(0.5)

    # 4. 保存结果（CSV + JSON）
    if all_danmu:
        df = pd.DataFrame(all_danmu)
        df.sort_values(by="offset_time_sec", inplace=True)
        df.reset_index(drop=True, inplace=True)

        desktop = os.path.expanduser("~/Desktop")
        csv_path = os.path.join(desktop, f"{vid}_full_danmu_sorted.csv")
        json_path = os.path.join(desktop, f"{vid}_danmu_raw.json")

        # 保存 CSV 文件
        df.to_csv(csv_path, index=False, encoding="utf-8-sig")

        # 保存 JSON 文件
        with open(json_path, "w", encoding="utf-8") as f:
            json.dump(all_danmu, f, ensure_ascii=False, indent=2)

        print(f"\n🎉 共抓取 {len(df)} 条弹幕：")
        print(f"📄 CSV 文件已保存到：{csv_path}")
        print(f"📘 JSON 文件已保存到：{json_path}")
    else:
        print("⚠️ 未抓取到任何弹幕。")


# ================== 启动执行 ==================
if __name__ == "__main__":
    crawl_danmu(cover_id, vid, video_path)


⚠️ 无法获取视频时长: could not convert string to float: ''
⚠️ 使用默认时长 10 分钟。
🔍 正在测试可用接口 ...
✅ 找到可用接口: https://dm.video.qq.com/barrage/segment/{vid}/t/v1/{start}/{end}

🚀 开始抓取弹幕 (step=30000 ms) ...
✅ 抓取第 0 段，共 507 条
✅ 抓取第 1 段，共 545 条
✅ 抓取第 2 段，共 529 条
✅ 抓取第 3 段，共 326 条
✅ 抓取第 4 段，共 357 条
✅ 抓取第 5 段，共 463 条
✅ 抓取第 6 段，共 570 条
✅ 抓取第 7 段，共 377 条
✅ 抓取第 8 段，共 404 条
✅ 抓取第 9 段，共 419 条
✅ 抓取第 10 段，共 353 条
✅ 抓取第 11 段，共 305 条
✅ 抓取第 12 段，共 404 条
✅ 抓取第 13 段，共 366 条
✅ 抓取第 14 段，共 518 条
✅ 抓取第 15 段，共 429 条
✅ 抓取第 16 段，共 185 条
✅ 抓取第 17 段，共 208 条


## Step 2：腾讯视频字幕

In [1]:
# 通过OCR方法拿到腾讯视频字幕
import os
import sys
import time
import re
import csv
import subprocess
import tempfile
from pathlib import Path

import mss
from PIL import Image, ImageOps, ImageFilter
import pytesseract
from pytesseract import Output

# ================== 你的腾讯视频链接 ==================
TENCENT_URL = "https://v.qq.com/x/cover/mzc00200li7mfpr/v41013vxrfa.html"

# ================== 全局配置 ==================

MAX_SECONDS = 4 * 60        # ★ 总采集时长：4 分钟
CAPTURE_FPS = 2             # 每秒采样次数

SKIP_FIRST_SECONDS = 25     # 前 25 秒一律丢掉（开头广告等）

PLAYWRIGHT_ALIVE_EXTRA = 10  # 浏览器额外存活时间（秒）

# ☆ 字幕所在大致区域（相对屏幕高度/宽度）
#   - 纵向：底部一条带，紧贴进度条上方
#   - 横向：只取中间 40% 宽度，避开左侧会员广告 & 右下角时间
SUBTITLE_TOP_RATIO    = 0.75
SUBTITLE_BOTTOM_RATIO = 0.90
SUBTITLE_LEFT_RATIO   = 0.30
SUBTITLE_RIGHT_RATIO  = 0.70

# OCR 相关（chi_sim 要装在默认 tessdata 目录）
TESSERACT_EXE = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
OCR_LANG      = "chi_sim"
BIN_THRESHOLD = 180          # 二值化阈值，可 160~200 微调
MIN_LINE_CHARS = 2
MERGE_MAX_GAP  = 0.8         # 同一条字幕允许的最大时间间隙（秒）

# 输出位置
DESKTOP = Path.home() / "Desktop"
OUT_CSV = DESKTOP / "captions.csv"
OUT_SRT = DESKTOP / "captions.srt"

# 调试截图
DEBUG_SAVE_IMAGES = True
DEBUG_SAVE_LIMIT  = 8
DEBUG_DIR         = DESKTOP / "caption_debug_screen"


# ================== 工具函数 ==================

def sec_to_srt(t: float) -> str:
    if t < 0:
        t = 0.0
    h = int(t // 3600)
    m = int((t % 3600) // 60)
    s = int(t % 60)
    ms = int(round((t - int(t)) * 1000))
    return f"{h:02d}:{m:02d}:{s:02d},{ms:03d}"


def edit_distance(a: str, b: str) -> int:
    if a == b:
        return 0
    la, lb = len(a), len(b)
    if la == 0:
        return lb
    if lb == 0:
        return la
    dp = [[0] * (lb + 1) for _ in range(la + 1)]
    for i in range(la + 1):
        dp[i][0] = i
    for j in range(lb + 1):
        dp[0][j] = j
    for i in range(1, la + 1):
        for j in range(1, lb + 1):
            cost = 0 if a[i-1] == b[j-1] else 1
            dp[i][j] = min(
                dp[i-1][j] + 1,
                dp[i][j-1] + 1,
                dp[i-1][j-1] + cost,
            )
    return dp[la][lb]


def similar_text(a: str, b: str, max_ratio: float = 0.2) -> bool:
    if not a or not b:
        return False
    dist = edit_distance(a, b)
    norm = dist / max(len(a), len(b))
    return norm <= max_ratio


def merge_segments(segments, max_gap=0.6):
    if not segments:
        return []
    segments.sort(key=lambda x: x[0])
    out = [list(segments[0])]
    for s, e, tx in segments[1:]:
        ps, pe, ptx = out[-1]
        if (s - pe) <= max_gap and similar_text(tx, ptx):
            out[-1][1] = max(pe, e)
            if len(tx) > len(ptx):
                out[-1][2] = tx
        else:
            out.append([s, e, tx])
    return [tuple(x) for x in out]


def ensure_ocr_ready():
    pytesseract.pytesseract.tesseract_cmd = TESSERACT_EXE
    os.environ.pop("TESSDATA_PREFIX", None)
    try:
        langs = pytesseract.get_languages(config="")
        print("Tesseract 可用语言：", langs)
        if OCR_LANG not in langs:
            print("⚠️ chi_sim 不在列表中，说明中文语言包还没安装好。")
    except Exception as e:
        print("⚠️ 语言列表检测失败：", e)


def binarize(img, threshold=BIN_THRESHOLD):
    """增强 + 放大 + 二值化，提高小字幕识别率。"""
    g = img.convert("L")
    g = ImageOps.autocontrast(g)
    g = g.filter(ImageFilter.MedianFilter(3))
    g = g.resize((g.width * 2, g.height * 2), Image.BICUBIC)
    bw = g.point(lambda p: 255 if p > threshold else 0, mode="1")
    return bw


def has_chinese(s: str) -> bool:
    return any("\u4e00" <= ch <= "\u9fff" for ch in s)


def chinese_char_count(s: str) -> int:
    return sum(1 for ch in s if "\u4e00" <= ch <= "\u9fff")


def extract_center_subtitle(crop_img: Image.Image) -> str | None:
    """
    在截下来的横条里，用 image_to_data 只取
    “最接近条带垂直中心的那一行中文” 作为字幕。
    """
    bw = binarize(crop_img, BIN_THRESHOLD)
    cfg = "--psm 6 --oem 3"
    data = pytesseract.image_to_data(
        bw, lang=OCR_LANG, config=cfg, output_type=Output.DICT
    )

    n = len(data["text"])
    h = bw.height
    center_y = h / 2.0

    # 按 line_num 聚合
    line_dict = {}
    for i in range(n):
        txt = data["text"][i].strip()
        if not txt:
            continue
        try:
            conf = int(data["conf"][i])
        except ValueError:
            conf = 0
        if conf < 60:
            continue
        if not has_chinese(txt):
            continue

        line_no = data["line_num"][i]
        left = data["left"][i]
        top = data["top"][i]
        height = data["height"][i]
        y_mid = top + height / 2.0

        line_dict.setdefault(line_no, []).append(
            dict(text=txt, x=left, y=y_mid)
        )

    if not line_dict:
        return None

    # 选：离条带中心最近、且汉字数量最多的那一行
    best_line = None
    best_score = None
    for line_no, items in line_dict.items():
        ys = [it["y"] for it in items]
        avg_y = sum(ys) / len(ys)
        line_txt = "".join([it["text"] for it in sorted(items, key=lambda x: x["x"])])
        ch_cnt = chinese_char_count(line_txt)
        score = abs(avg_y - center_y) - ch_cnt * 2.0
        if best_score is None or score < best_score:
            best_score = score
            best_line = line_txt

    if not best_line:
        return None

    cleaned = re.sub(r"[^\u4e00-\u9fff，。！？：；、“”‘’…·0-9A-Za-z]", "", best_line)
    if len(cleaned) < MIN_LINE_CHARS:
        return None
    return cleaned


# ================== 子进程：Playwright 控制浏览器 ==================

def launch_browser_with_playwright():
    temp_dir = Path(tempfile.gettempdir())
    script_path = temp_dir / "tencent_playwright_driver_tmp.py"

    playwright_script = f"""\
from playwright.sync_api import sync_playwright
import time

URL = {TENCENT_URL!r}
PLAY_SECONDS = {MAX_SECONDS + PLAYWRIGHT_ALIVE_EXTRA}

def main():
    with sync_playwright() as p:
        try:
            browser = p.chromium.launch(
                headless=False,
                channel="chrome",
                args=["--disable-blink-features=AutomationControlled"],
            )
        except Exception:
            browser = p.chromium.launch(
                headless=False,
                args=["--disable-blink-features=AutomationControlled"],
            )

        ctx = browser.new_context(
            viewport={{"width": 1366, "height": 820}},
            user_agent=(
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/120.0 Safari/537.36"
            ),
        )
        page = ctx.new_page()
        try:
            page.goto(URL, wait_until="domcontentloaded", timeout=90_000)
        except Exception:
            time.sleep(5)

        # 播放
        play_selectors = [
            'button[aria-label*="播放"]',
            'div[class*="txp_button_play"]',
            'div[class*="txp_btn"]',
            'button[class*="txp"]',
            'div.txp_btn_play',
        ]
        for sel in play_selectors:
            try:
                btn = page.query_selector(sel)
                if btn:
                    btn.click()
                    break
            except Exception:
                pass

        time.sleep(2)

        # 全屏
        fullscreen_selectors = [
            'button[aria-label*="全屏"]',
            'div[class*="txp_button_fullscreen"]',
            'div.txp_btn_fullscreen',
        ]
        for sel in fullscreen_selectors:
            try:
                b = page.query_selector(sel)
                if b:
                    b.click()
                    break
            except Exception:
                pass

        time.sleep(1)

        # 打开“语言”
        lang_selectors = [
            'text=语言',
            'button[aria-label*="语言"]',
            'div[aria-label*="语言"]',
            'span:has-text("语言")',
        ]
        for sel in lang_selectors:
            try:
                el = page.query_selector(sel)
                if el:
                    el.click()
                    time.sleep(0.5)
                    break
            except Exception:
                pass

        # 选择中文字幕
        subtitle_options = [
            'text=简体中文',
            'text=中文（简体）',
            'text=中文字幕',
            'text=中文',
        ]
        found_sub = False
        for sel in subtitle_options:
            try:
                item = page.query_selector(sel)
                if item:
                    item.click()
                    found_sub = True
                    break
            except Exception:
                pass

        if found_sub:
            print("✅ 已尝试开启中文字幕")
        else:
            print("⚠️ 未能自动找到字幕菜单项，可能仍需人工确认。")

        time.sleep(PLAY_SECONDS)
        ctx.close()
        browser.close()

if __name__ == "__main__":
    main()
"""
    script_path.write_text(playwright_script, encoding="utf-8")
    print("Playwright 子脚本路径:", script_path)
    proc = subprocess.Popen([sys.executable, str(script_path)])
    return proc


# ================== 主流程：屏幕截图 + OCR ==================

def capture_subtitles_from_screen():
    ensure_ocr_ready()

    segments = []
    active_text = None
    active_start = None

    DESKTOP.mkdir(parents=True, exist_ok=True)
    if DEBUG_SAVE_IMAGES:
        DEBUG_DIR.mkdir(parents=True, exist_ok=True)

    print("▶ 正在启动浏览器并自动播放视频、打开字幕...")
    browser_proc = launch_browser_with_playwright()

    # 给浏览器一点时间加载&进片头
    time.sleep(10)

    interval = 1.0 / max(0.1, CAPTURE_FPS)
    start_wall = time.time()
    debug_saved = 0

    with mss.mss() as sct:
        monitor = sct.monitors[1]

        screen_width = monitor["width"]
        screen_height = monitor["height"]
        screen_left = monitor["left"]
        screen_top = monitor["top"]

        print("屏幕尺寸:", screen_width, "x", screen_height)
        print("开始录制字幕...\n")

        while True:
            now = time.time()
            if now - start_wall > MAX_SECONDS:
                break

            t_sec = now - start_wall
            if t_sec < SKIP_FIRST_SECONDS:
                time.sleep(interval)
                continue

            shot = sct.grab(monitor)
            img = Image.frombytes("RGB", shot.size, shot.rgb)

            # 只截底部中间一条带
            crop_left   = int(screen_left + screen_width  * SUBTITLE_LEFT_RATIO)
            crop_right  = int(screen_left + screen_width  * SUBTITLE_RIGHT_RATIO)
            crop_top    = int(screen_top  + screen_height * SUBTITLE_TOP_RATIO)
            crop_bottom = int(screen_top  + screen_height * SUBTITLE_BOTTOM_RATIO)

            crop = img.crop((
                crop_left   - screen_left,
                crop_top    - screen_top,
                crop_right  - screen_left,
                crop_bottom - screen_top,
            ))

            if DEBUG_SAVE_IMAGES and debug_saved < DEBUG_SAVE_LIMIT:
                crop.save(DEBUG_DIR / f"strip_{debug_saved:03d}.png")
                debug_saved += 1

            try:
                final_txt = extract_center_subtitle(crop)
            except pytesseract.TesseractError as e:
                print("❌ Tesseract 调用失败：", e)
                break

            if final_txt:
                if active_text is None:
                    active_text = final_txt
                    active_start = t_sec
                else:
                    if similar_text(final_txt, active_text):
                        # 同一句字幕，持续时间拉长
                        pass
                    else:
                        # 新的一句字幕，先收尾上一句
                        if active_start is not None:
                            end_t = t_sec
                            segments.append(
                                (active_start,
                                 max(active_start + 0.5, end_t),
                                 active_text)
                            )
                        active_text = final_txt
                        active_start = t_sec
            else:
                if active_text is not None and active_start is not None:
                    end_t = t_sec
                    segments.append(
                        (active_start,
                         max(active_start + 0.5, end_t),
                         active_text)
                    )
                    active_text = None
                    active_start = None

            time.sleep(interval)

    try:
        browser_proc.terminate()
    except Exception:
        pass

    if active_text is not None and active_start is not None:
        end_t = time.time() - start_wall
        segments.append(
            (active_start, max(active_start + 0.5, end_t), active_text)
        )

    segments = merge_segments(segments, max_gap=MERGE_MAX_GAP)

    rows = []
    for i, (s, e, tx) in enumerate(segments, 1):
        rows.append(
            dict(
                index=i,
                start_sec=round(s, 3),
                end_sec=round(e, 3),
                duration=round(max(0.0, e - s), 3),
                text=tx,
            )
        )

    # CSV：gbk 方便 Excel
    with open(OUT_CSV, "w", encoding="gbk", newline="") as f:
        w = csv.DictWriter(
            f, fieldnames=["index", "start_sec", "end_sec", "duration", "text"]
        )
        w.writeheader()
        for r in rows:
            w.writerow(r)

    # SRT：utf-8-sig
    with open(OUT_SRT, "w", encoding="utf-8-sig") as f:
        for i, (s, e, tx) in enumerate(segments, 1):
            f.write(str(i) + "\n")
            f.write(f"{sec_to_srt(s)} --> {sec_to_srt(e)}\n")
            lines = [t for t in re.split(r"[\n\r]+", tx) if t.strip()]
            if len(lines) > 2:
                lines = lines[:2]
            f.write("\n".join(lines) + "\n\n")

    print("\n✅ 录制完成")
    print("CSV:", OUT_CSV)
    print("SRT:", OUT_SRT)
    print(f"共 {len(segments)} 条字幕段（前 5 条预览）：")
    for r in rows[:5]:
        print(r["start_sec"], "→", r["end_sec"], ":", r["text"])


# 真正执行
capture_subtitles_from_screen()


Tesseract 可用语言： ['chi_sim', 'eng', 'osd']
▶ 正在启动浏览器并自动播放视频、打开字幕...
Playwright 子脚本路径: C:\Users\koto\AppData\Local\Temp\tencent_playwright_driver_tmp.py
屏幕尺寸: 2880 x 1800
开始录制字幕...



KeyboardInterrupt: 

## Step 3：数据初步清洗——whisper验证、文本对齐

In [ ]:
# 这是Whisper验证
import os
import json
import pandas as pd
import numpy as np
import warnings
from pathlib import Path
from snownlp import SnowNLP

# ========== 配置区域 ==========
USE_SEMANTIC_ALIGNMENT = True  # 是否启用语义对齐（需要 sentence-transformers）
vid = "v41013vxrfa"            # 视频ID
desktop = Path(os.path.expanduser("~/Desktop"))
segments_path = desktop / "segments.json"               # Whisper 输出
danmu_path = desktop / f"{vid}_danmu_raw.json"          # 弹幕原始数据
out_csv = desktop / f"{vid}_danmu_aligned.csv"          # 输出CSV
out_json = desktop / f"{vid}_danmu_aligned.json"        # 输出JSON
# =============================


# ========== 载入数据 ==========
if not segments_path.exists():
    raise FileNotFoundError(f"❌ 找不到 Whisper 语音识别结果文件: {segments_path}")
if not danmu_path.exists():
    raise FileNotFoundError(f"❌ 找不到弹幕文件: {danmu_path}")

segments = json.load(open(segments_path, encoding='utf-8'))
danmu_raw = json.load(open(danmu_path, encoding='utf-8'))
df = pd.DataFrame(danmu_raw)

print(f"📦 弹幕载入：{len(df)} 条")
print(f"🎧 语音片段：{len(segments)} 段")

# ========== 基础时间匹配 ==========
def coarse_match_text(t, segs):
    """根据时间粗匹配 Whisper 语音片段"""
    for s in segs:
        if s['start'] <= t <= s['end']:
            return s['text']
    return ''

df['matched_text'] = df['offset_time_sec'].apply(lambda t: coarse_match_text(t, segments))
df['aligned_time'] = df['offset_time_sec']
df['aligned_text'] = df['matched_text']
df['similarity'] = np.nan


# ========== 语义对齐 ==========
if USE_SEMANTIC_ALIGNMENT:
    try:
        from sentence_transformers import SentenceTransformer, util
        print("🧠 正在进行语义对齐 ...（加载模型中）")

        sim_model = SentenceTransformer('shibing624/text2vec-base-chinese')
        seg_texts = [s['text'] for s in segments]
        emb_segments = sim_model.encode(seg_texts, convert_to_tensor=True, show_progress_bar=True)

        def sem_align(text):
            """语义相似度匹配"""
            if not text.strip():
                return pd.Series([np.nan, '', 0.0])
            e = sim_model.encode(text, convert_to_tensor=True)
            scores = util.cos_sim(e, emb_segments)[0]
            best_idx = int(np.argmax(scores))
            best_seg = segments[best_idx]
            return pd.Series([best_seg['start'], best_seg['text'], float(scores[best_idx])])

        df[['aligned_time', 'aligned_text', 'similarity']] = df['content'].apply(
            lambda x: sem_align(str(x))
        )

        # 对齐失败回退到粗匹配
        mask = df['aligned_time'].isna()
        df.loc[mask, 'aligned_time'] = df.loc[mask, 'offset_time_sec']
        df.loc[mask, 'aligned_text'] = df.loc[mask, 'matched_text']

        print("✅ 语义对齐完成")
    except Exception as e:
        warnings.warn(f"⚠️ 语义对齐跳过：{e}")
else:
    print("🚫 已禁用语义对齐，仅使用时间粗匹配。")


# ========== 情感分析 ==========
print("🎭 正在进行情感分析 ...")
df['sentiment'] = df['content'].apply(lambda x: round(SnowNLP(str(x)).sentiments, 3))
df['minute'] = (df['aligned_time'] // 60).astype(int)
print("✅ 情感分析完成")

# ========== 输出结果 ==========
df.to_csv(out_csv, index=False, encoding="utf-8-sig")
with open(out_json, "w", encoding="utf-8") as f:
    json.dump(df.to_dict(orient="records"), f, ensure_ascii=False, indent=2)

print("\n🎉 全部完成！")
print(f"📄 对齐与情感分析结果 (CSV)：{out_csv}")
print(f"📘 对齐与情感分析结果 (JSON)：{out_json}")


## Step 4：数据深度清洗——删除无用栏、集合

In [ ]:
# -*- coding: utf-8 -*-
# 合并 talkshow 12 集（含第 12 集上中下）所有 *_timealigned_view.csv 为一张“长表”
# 作者：你的研究助手
# 需求依赖：pip install pandas numpy

import os, re, glob, json
import numpy as np
import pandas as pd

# ------------ 1) 配置 ------------
BASE_DIR = r"C:\Users\koto\Desktop\dataset"     # 你的数据文件夹
OUT_DIR  = os.path.join(BASE_DIR, "talkshow_dataset")
os.makedirs(OUT_DIR, exist_ok=True)

# 只合并 *_timealigned_view.csv（更干净）
PATTERNS = ["*timealigned_view.csv", "*timealign*_view.csv"]

# ------------ 2) 通用工具 ------------
def try_read_csv(path):
    """多编码容错读取，优先 utf-8-sig -> utf-8 -> gb18030"""
    last_err = None
    for enc in ["utf-8-sig", "utf-8", "gb18030"]:
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception as e:
            last_err = e
    raise last_err

def norm(s):
    return str(s).strip().lower().replace("_"," ").replace("-"," ")

def find_col(df, target, contains_ok=True):
    """模糊找列名，支持 content / mapped text / mapped_text 等变体"""
    t = norm(target)
    for c in df.columns:
        if norm(c) == t:
            return c
    if contains_ok:
        for c in df.columns:
            if t in norm(c):
                return c
    return None

def to_seconds(x):
    if pd.isna(x): return np.nan
    s = str(x).strip()
    # 已经是数值
    try:
        return float(s)
    except:
        pass
    # 支持 mm:ss / hh:mm:ss
    parts = s.split(":")
    try:
        if len(parts)==3:
            h,m,rest = parts
            return int(h)*3600 + int(m)*60 + float(rest)
        elif len(parts)==2:
            m,rest = parts
            return int(m)*60 + float(rest)
    except:
        return np.nan
    return np.nan

def parse_ep_and_part(fname):
    """
    从文件名提取集数与分段：
    - talkshow01s_... -> ep=1, part='s'
    - talkshow12m_... -> ep=12, part='m'
    - 若缺少分段则返回空字符串
    """
    base = os.path.basename(fname)
    m = re.search(r"talkshow(\d{2})([smx])?_.*timealign.*_view\.csv", base, flags=re.IGNORECASE)
    if not m:
        # 兼容更宽松的情况
        m = re.search(r"talkshow(\d{2})([smx])?", base, flags=re.IGNORECASE)
    ep_num = int(m.group(1)) if m else -1
    part   = (m.group(2) or "").lower() if m else ""
    part_cn = {"s":"上", "m":"中", "x":"下"}.get(part, "")
    return ep_num, part, part_cn, base

# ------------ 3) 扫描文件 ------------
csv_paths = []
for pat in PATTERNS:
    csv_paths += glob.glob(os.path.join(BASE_DIR, pat))
csv_paths = sorted(set(csv_paths))

if not csv_paths:
    raise SystemExit(f"没有找到 *_timealign*_view.csv，目录：{BASE_DIR}")

print(f"发现 {len(csv_paths)} 个文件：")
for p in csv_paths:
    print(" -", os.path.basename(p))

# ------------ 4) 逐文件读取并标准化 ------------
merged_rows = []
summary_rows = []

for path in csv_paths:
    df = try_read_csv(path)

    # 找关键列（不同版本可能叫法略不同）
    c_content  = find_col(df, "content")
    c_dm_mmss  = find_col(df, "offset_time_mmss")
    c_sub_text = find_col(df, "mapped text") or find_col(df, "mapped_text")
    c_sub_mmss = find_col(df, "mapped_time_mmss")

    need = [c_content, c_dm_mmss, c_sub_text, c_sub_mmss]
    if not all(need):
        print(f"[跳过] 列名不完整：{os.path.basename(path)}")
        continue

    # 统一重命名
    df_std = pd.DataFrame({
        "content":         df[c_content].astype(str),
        "offset_time_mmss":df[c_dm_mmss],
        "mapped_text":     df[c_sub_text].astype(str),
        "mapped_time_mmss":df[c_sub_mmss],
    })

    # 秒数列（若原表已有 *_sec 也没关系，这里统一计算，确保一致）
    df_std["dm_time_s"]  = df_std["offset_time_mmss"].apply(to_seconds)
    df_std["sub_time_s"] = df_std["mapped_time_mmss"].apply(to_seconds)

    # 标注集数/分段
    ep_num, part, part_cn, base = parse_ep_and_part(path)
    df_std["ep_num"]   = ep_num
    df_std["ep_label"] = f"EP{ep_num:02d}" if ep_num>0 else "EP??"
    df_std["ep_part"]  = part              # s/m/x
    df_std["ep_part_cn"] = part_cn         # 上/中/下（第12集会用到）

    # 记录
    merged_rows.append(df_std)
    summary_rows.append({
        "file": base,
        "ep_num": ep_num,
        "ep_label": f"EP{ep_num:02d}" if ep_num>0 else "EP??",
        "ep_part": part,
        "ep_part_cn": part_cn,
        "rows": len(df_std)
    })

# 合并
if not merged_rows:
    raise SystemExit("未有可合并的数据，请检查列名或文件格式。")
merged = pd.concat(merged_rows, ignore_index=True)

# 去除明显重复（同一集/分段，同一弹幕和字幕时间&文本完全一致）
merged = merged.drop_duplicates(subset=[
    "ep_num","ep_part","dm_time_s","sub_time_s","content","mapped_text"
]).reset_index(drop=True)

# ------------ 5) 导出 ------------
out_csv = os.path.join(OUT_DIR, "talkshow_dataset.csv")
merged.to_csv(out_csv, index=False, encoding="utf-8-sig")

summary = pd.DataFrame(summary_rows).sort_values(["ep_num","ep_part"])
summary_out = os.path.join(OUT_DIR, "intro_talkshow_dataset.csv")
summary.to_csv(summary_out, index=False, encoding="utf-8-sig")

# 简要统计打印
print("\n合并完成！")
print("输出文件：", out_csv)
print("汇总清单：", summary_out)
print("\n各集行数预览：")
print(summary.groupby(["ep_label","ep_part"]).agg(rows=("rows","sum")).reset_index().head(20))

# 如果你想确认第12集的三段是否都在：
if (summary['ep_num']==12).any():
    print("\n第 12 集分段统计：")
    print(summary[summary['ep_num']==12][["file","ep_part","ep_part_cn","rows"]])


## Step 5：数据可视化

In [ ]:
# ======================= 0) （可选）自动安装缺失依赖 =======================
import sys, subprocess, importlib
def _ensure(pkgs):
    for p in pkgs:
        try: importlib.import_module(p.split("==")[0])
        except Exception:
            subprocess.check_call([sys.executable, "-m", "pip", "install", p, "-q"])
_ensure(["pandas","numpy","plotly","ipywidgets","pyarrow"])  # opencc 可选

# ======================= 1) 导入与全局配置 =======================
import os, re, math, unicodedata
import numpy as np
import pandas as pd
from collections import Counter

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

pd.options.mode.copy_on_write = True
pio.renderers.default = "notebook_connected"  # VSCode/Jupyter 均可
pio.templates.default = "simple_white"

# --------- 路径与字体 ---------
DATA_PATH           = r"C:\Users\koto\Desktop\talkshow_dataset.csv"
CACHE_PARQUET       = r"C:\Users\koto\Desktop\talkshow_cache_fast.parquet"
FONT_FAMILY         = "Microsoft YaHei, SimHei, Arial"

# 你提供的成品 HTML（如在 Linux/云端，请改为实际路径）
HTML_HEATMAP_PATH   = r"/mnt/data/danmu_heatmap.html"
HTML_WORDCLOUD_PATH = r"/mnt/data/interactive_sector_wordcloud(1).html"

# --------- 极速预览参数（后期出最终图再调高精度） ---------
FAST_PREVIEW       = True     # 最终版改为 False
USE_OPENCC         = False    # 需要繁→简时设 True（需安装 opencc）
BIN_SEC_DEFAULT    = 8        # 时间线分箱（秒）
WIN_SEC_DEFAULT    = 4        # 峰值窗口 ± 秒
HEATMAP_PCT_BINS   = 36       # 0-100% 进度热力图横向格
BASE_SAMPLE        = 4000     # 全局对照抽样上限（越大越慢）
TOP_N_TERMS        = 12       # 关键词展示条数

# --------- 统一主题色 ---------
THEME = dict(
    line="#2F5597",    # 时间线
    marker="#F18F01",  # 峰值点
    vline="#E74C3C",   # 播放线
    heat_seq=px.colors.sequential.YlGnBu,  # 热力图
    pos="#2ECC71",
    neg="#E74C3C",
    conf="#7F8C8D",
    app="#9B59B6",
    text="#222222",
)

# ======================= 2) 读数 & 预处理（带缓存） =======================
def read_csv_ch(path):
    for enc in ["utf-8-sig","utf-8","gb18030"]:
        try: return pd.read_csv(path, encoding=enc)
        except Exception: pass
    return pd.read_csv(path, encoding="utf-8", errors="ignore")

def to_seconds(x):
    if pd.isna(x): return np.nan
    s=str(x).strip()
    try: return float(s)
    except Exception: pass
    p=s.split(":")
    try:
        if len(p)==3: return int(p[0])*3600+int(p[1])*60+float(p[2])
        if len(p)==2: return int(p[0])*60+float(p[1])
    except Exception: return np.nan
    return np.nan

def to_simplified(s):
    s = unicodedata.normalize("NFKC", str(s))
    if USE_OPENCC:
        try:
            from opencc import OpenCC
            s = OpenCC("t2s").convert(s)
        except Exception: pass
    s = re.sub(r"[\u200b-\u200f\u202a-\u202e]", "", s)
    return s

def pick_col(df, name):
    tgt = name.lower().replace("_"," ")
    for c in df.columns:
        cc = c.lower().replace("_"," ")
        if cc==tgt or tgt in cc: return c
    return None

def load_data(csv_path, cache_path):
    if os.path.exists(cache_path):
        return pd.read_parquet(cache_path)

    raw = read_csv_ch(csv_path)
    c_content  = pick_col(raw, "content")
    c_dm_mmss  = pick_col(raw, "offset_time_mmss")
    c_sub_text = pick_col(raw, "mapped text") or pick_col(raw,"mapped_text")
    c_sub_mmss = pick_col(raw, "mapped_time_mmss")
    c_ep_num   = pick_col(raw, "ep_num")

    assert all([c_content,c_dm_mmss,c_sub_text,c_sub_mmss]), \
        "缺少必须列：content / offset_time_mmss / mapped_text / mapped_time_mmss"

    df = pd.DataFrame({
        "content":     raw[c_content].astype(str),
        "dm_time_s":   raw[c_dm_mmss].apply(to_seconds),
        "mapped_text": raw[c_sub_text].astype(str),
        "sub_time_s":  raw[c_sub_mmss].apply(to_seconds),
        "ep_num":      (raw[c_ep_num] if c_ep_num else 1),
    })
    df["ep_num"]   = df["ep_num"].fillna(1).astype(int)
    df["ep_label"] = df["ep_num"].map(lambda x: f"EP{x:02d}")
    df = df.dropna(subset=["dm_time_s","sub_time_s"]).reset_index(drop=True)

    if USE_OPENCC:
        df["content"]     = df["content"].map(to_simplified)
        df["mapped_text"] = df["mapped_text"].map(to_simplified)

    # 情绪/笑类正则（向量化）
    def _re(pats): return re.compile("|".join(pats), re.IGNORECASE)
    LAUGH_RE    = _re([r"(哈){2,}", r"(呵){2,}", r"笑(死|疯|翻|裂|喷|哭)?", r"233+", r"hhh+", r"xswl", r"乐(死|疯|坏)"])
    POSITIVE_RE = _re([r"好笑", r"绝了", r"太?逗了", r"有梗", r"高能", r"厉害", r"天才"])
    NEGATIVE_RE = _re([r"不好笑", r"无聊", r"烂", r"差劲", r"冷场", r"尴尬", r"拉胯", r"低俗"])
    CONFUSE_RE  = _re([r"\?{2,}", r"啥意思", r"没懂", r"不懂"])
    APPLAUSE_RE = _re([r"鼓掌", r"掌声", r"拍手", r"respect"])

    s = df["content"]
    df["is_laugh"]    = s.str.contains(LAUGH_RE,    regex=True).astype(int)
    df["is_positive"] = s.str.contains(POSITIVE_RE, regex=True).astype(int)
    df["is_negative"] = s.str.contains(NEGATIVE_RE, regex=True).astype(int)
    df["is_confuse"]  = s.str.contains(CONFUSE_RE,  regex=True).astype(int)
    df["is_applause"] = s.str.contains(APPLAUSE_RE, regex=True).astype(int)
    df["lag_s"]       = df["dm_time_s"] - df["sub_time_s"]

    # 瘦身后缓存
    keep = ["content","dm_time_s","mapped_text","sub_time_s","ep_num","ep_label",
            "is_laugh","is_positive","is_negative","is_confuse","is_applause","lag_s"]
    df = df[keep]
    try: df.to_parquet(cache_path, index=False)
    except Exception: pass
    return df

assert os.path.exists(DATA_PATH), f"找不到数据：{DATA_PATH}"
df = load_data(DATA_PATH, CACHE_PARQUET)

EP_OPTS   = sorted(df["ep_num"].unique().tolist())
EP_LABELS = dict(df[["ep_num","ep_label"]].drop_duplicates().values.tolist())

# ======================= 3) 统计 & 可视化工具 =======================
def laughing_signal(frame):
    # 全局统一的“加权笑指数”
    return (frame["is_laugh"] 
            + 0.5*frame["is_positive"] 
            + 0.7*frame["is_applause"] 
            - 0.4*frame["is_negative"] 
            - 0.3*frame["is_confuse"])

def bin_series(g, bin_sec, metric="weighted"):
    if g.empty: return pd.DataFrame(columns=["t","count"])
    tbin = (g["dm_time_s"]//bin_sec).astype(int)
    if metric=="weighted":
        vals = laughing_signal(g).groupby(tbin).sum()
    else:
        vals = g.groupby(tbin)["is_laugh"].sum()
    return pd.DataFrame({"t": vals.index.values*bin_sec, "count": vals.values})

def gaussian_smooth(y, k=7):
    # k 为奇数更好；小于3直接返回
    k = int(max(3, k))
    if k % 2 == 0: k += 1
    if len(y) < k: return y
    xs = np.arange(k) - (k-1)/2
    sigma = max(1.0, k/2.5)
    kernel = np.exp(-0.5*(xs/sigma)**2)
    kernel /= kernel.sum()
    return np.convolve(y, kernel, mode="same")

def find_peaks(x, y, top_n=3, min_dist_bins=3, min_prom=0.0):
    # 先找局部极大，再按 prominence 排序，最后做最小间距筛选
    n = len(y)
    if n < 3: return []
    # 候选
    cand = []
    for i in range(1, n-1):
        if y[i] >= y[i-1] and y[i] >= y[i+1] and y[i] > 0:
            # 左右最小值
            li = i-1
            mleft = y[i]
            while li >= 0 and y[li] <= y[li+1]:
                mleft = min(mleft, y[li]); li -= 1
            ri = i+1
            mright = y[i]
            while ri < n and y[ri] <= y[ri-1]:
                mright = min(mright, y[ri]); ri += 1
            prom = y[i] - max(mleft, mright)
            if prom >= min_prom:
                cand.append((i, y[i], prom))
    # prominence优先，其次高度
    cand.sort(key=lambda t: (t[2], t[1]), reverse=True)
    chosen = []
    for i, h, prom in cand:
        xi = x[i]
        # 距离筛选（以 bin 个数为单位）
        ok = True
        for cx, ch in chosen:
            if abs(i - np.searchsorted(x, cx)) < min_dist_bins:
                ok = False; break
        if ok:
            chosen.append((xi, h))
        if len(chosen) >= max(1, top_n):
            break
    return chosen

STOP = set(list("哈哈呵呵啊呀哦恩嗯这这个那个一个什么怎么就是可以真的然后所以不会不是我们你们他们而且但是还有啦吧嘛呢"))
def tokenize_bi(s):
    s = re.sub(r"[^\u4e00-\u9fffA-Za-z0-9]+","", str(s))
    toks = re.findall(r'(?=([\u4e00-\u9fff]{2}))', s)  # lookahead 二字词
    return [w for w in toks if (w not in STOP) and (w[0]!=w[1])]

def keywords_peak_vs_base(df_ep, peak_center, win_sec, topN=TOP_N_TERMS, base_sample=BASE_SAMPLE):
    win = df_ep[(df_ep["sub_time_s"]>=peak_center-win_sec) & (df_ep["sub_time_s"]<=peak_center+win_sec)]
    mask = (df_ep["is_laugh"]|df_ep["is_positive"]|df_ep["is_applause"]).astype(bool)
    base = df_ep[mask & ~df_ep.index.isin(win.index)]
    if len(base)>base_sample:
        base = base.sample(base_sample, random_state=42)

    cp, cb = Counter(), Counter()
    for s in win["content"]:  cp.update(tokenize_bi(s))
    for s in base["content"]: cb.update(tokenize_bi(s))

    eps=1e-9; Np=sum(cp.values())+eps; Nb=sum(cb.values())+eps
    rows=[(t, math.log((c/Np+eps)/(cb.get(t,0)/Nb+eps))) for t,c in cp.items()]
    rows.sort(key=lambda x:x[1], reverse=True)
    return pd.DataFrame(rows[:topN], columns=["term","score"])

def aggregate_subs(df_ep, center, win_sec):
    sub = df_ep[(df_ep["sub_time_s"]>=center-win_sec) & (df_ep["sub_time_s"]<=center+win_sec)].copy()
    g = sub.groupby(["mapped_text","sub_time_s"], as_index=False).agg(
        win_laugh=("is_laugh","sum"),
        win_pos=("is_positive","sum"),
        win_neg=("is_negative","sum"),
        win_conf=("is_confuse","sum"),
        win_app=("is_applause","sum"),
        median_lag=("lag_s","median")
    )
    g["laugh_score"] = g["win_laugh"] + 0.5*g["win_pos"] + 0.7*g["win_app"] \
                       - 0.8*g["win_neg"] - 0.5*g["win_conf"]
    return g.sort_values("laugh_score", ascending=False)

def heatmap_percent(df_all, pct_bins=HEATMAP_PCT_BINS, metric="weighted", basis="dm", norm="raw"):
    # basis: "dm" 使用 dm_time_s；"sub" 使用 sub_time_s
    tcol = "dm_time_s" if basis=="dm" else "sub_time_s"
    out=[]
    labels=[]
    for ep, g in df_all.groupby("ep_num"):
        g = g.sort_values(tcol)
        t0, t1 = g[tcol].min(), g[tcol].max()
        dur = max(t1 - t0, 1e-6)
        gg = g.copy()
        gg["pct_bin"] = ((gg[tcol]-t0)/dur * pct_bins).clip(0,pct_bins-1).astype(int)
        if metric=="weighted":
            vals = laughing_signal(gg)
        else:
            vals = gg["is_laugh"]
        cnt = vals.groupby(gg["pct_bin"]).sum()
        row = np.array([float(cnt.get(i,0.0)) for i in range(pct_bins)], dtype=float)
        if norm=="row":
            m = row.max()
            row = row/m if m>0 else row
        out.append(row); labels.append(f"EP{ep:02d}")
    if not out:
        return np.zeros((0, pct_bins)), []
    mat = np.vstack(out)
    return mat, labels

# ======================= 4) 交互控件（ipywidgets） =======================
dd_ep     = widgets.Dropdown(options=[(EP_LABELS.get(e,f"EP{e:02d}"),e) for e in EP_OPTS], value=EP_OPTS[0], description="集数")
sl_bin    = widgets.IntSlider(min=2,max=20,step=1,value=BIN_SEC_DEFAULT,description="分箱(s)")
sl_win    = widgets.IntSlider(min=2,max=12, step=1,value=WIN_SEC_DEFAULT,description="窗口±s")
sl_peak   = widgets.IntSlider(min=1,max=8, step=1,value=3, description="峰值数")
sl_smooth = widgets.IntSlider(min=3,max=21,step=2,value=7, description="平滑窗")
sl_prom   = widgets.FloatSlider(min=0.0,max=5.0,step=0.1,value=0.5, description="显著性≥")
sl_mdist  = widgets.IntSlider(min=1,max=10,step=1,value=3, description="最小间距")
tg_metric = widgets.ToggleButtons(options=[("纯笑","raw"),("加权","weighted")], value="weighted", description="指标")
tg_basis  = widgets.ToggleButtons(options=[("弹幕时间","dm"),("字幕时间","sub")], value="dm", description="时间基准")
dd_hnorm  = widgets.Dropdown(options=[("原始","raw"),("行归一化(0-1)","row")], value="raw", description="热力图")

play      = widgets.Play(value=0, min=0, max=1000, step=10, interval=400, description="Play")
sl_play   = widgets.FloatSlider(min=0,max=1,step=0.001,value=0, readout=False)
widgets.jslink((play,"value"), (sl_play,"value"))

# 输出区
out_timeline = widgets.Output()
out_kw       = widgets.Output()
out_subs     = widgets.Output()
out_table    = widgets.Output()
out_heat     = widgets.Output()
out_caption  = widgets.Output()
out_html1    = widgets.Output()
out_html2    = widgets.Output()

def render_html_file(path, out):
    with out:
        clear_output(wait=True)
        try:
            if path and os.path.exists(path):
                html = open(path, "r", encoding="utf-8").read()
                display(HTML(html))
            else:
                display(HTML(f"<div style='font-family:{FONT_FAMILY}'>找不到 HTML：<code>{path}</code></div>"))
        except Exception as e:
            display(HTML(f"<div style='font-family:{FONT_FAMILY};color:#b00'>HTML 加载失败：{e}</div>"))

# ======================= 5) 渲染函数 =======================
def render_all(*_):
    ep       = dd_ep.value
    bin_sec  = sl_bin.value
    win_sec  = sl_win.value
    n_peaks  = sl_peak.value
    k_smooth = sl_smooth.value
    min_prom = sl_prom.value
    min_dist = sl_mdist.value
    metric   = tg_metric.value
    basis    = tg_basis.value
    hnorm    = dd_hnorm.value
    playhead = sl_play.value

    g = df[df["ep_num"]==ep]
    s = bin_series(g, bin_sec, metric=metric)
    y = gaussian_smooth(s["count"].values, k=k_smooth); x=s["t"].values

    # 峰值（按 prominence & 最小间距），并选择离播放位置最近的一个用于解释
    peaks = find_peaks(x, y, top_n=n_peaks, min_dist_bins=min_dist, min_prom=min_prom)
    if len(g)>0:
        tmin, tmax = g["dm_time_s"].min(), g["dm_time_s"].max()
        cur_t = tmin + playhead*(tmax - tmin)
    else:
        cur_t = 0.0
    if peaks:
        peak_t = min(peaks, key=lambda p: abs(p[0]-cur_t))[0]
    else:
        peak_t = x[int(len(x)*0.5)] if len(x)>0 else 0.0

    # 1) 时间线
    with out_timeline:
        clear_output(wait=True)
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x, y=y, mode="lines", name="笑密度（平滑）",
                                 line=dict(color=THEME["line"], width=2)))
        for i,(tx,ty) in enumerate(peaks,1):
            fig.add_trace(go.Scatter(x=[tx], y=[ty], mode="markers+text",
                                     text=[f"#{i}"], textposition="top center",
                                     marker=dict(size=9, color=THEME["marker"]),
                                     name=f"峰值{i}"))
        fig.add_vline(x=cur_t, line=dict(color=THEME["vline"], width=2, dash="dot"))
        fig.update_layout(font=dict(family=FONT_FAMILY, color=THEME["text"]),
                          height=360, margin=dict(l=60,r=30,t=40,b=10),
                          legend=dict(orientation="h",y=1.02,x=0))
        fig.update_xaxes(title_text="时间（秒）")
        ylabel = "笑类强度（平滑, 加权）" if metric=="weighted" else "笑类弹幕条数（平滑）"
        fig.update_yaxes(title_text=ylabel, rangemode="tozero")
        display(fig)

    # 2) 峰值关键词（为什么笑） — 条形更可读
    kw = keywords_peak_vs_base(g, peak_t, win_sec, topN=TOP_N_TERMS, base_sample=BASE_SAMPLE)
    with out_kw:
        clear_output(wait=True)
        kshow = kw.sort_values("score")
        figkw = px.bar(kshow, x="score", y="term", orientation="h")
        figkw.update_traces(marker_color=THEME["marker"])
        figkw.update_layout(height=320, margin=dict(l=100,r=20,t=10,b=10),
                            font=dict(family=FONT_FAMILY))
        figkw.update_xaxes(title_text="显著性（峰值相对全局对数提升）")
        figkw.update_yaxes(title_text="")
        display(figkw)

    # 3) 字幕响应（对什么内容笑）— 按 laugh_score 着色
    subs = aggregate_subs(g, peak_t, win_sec)
    with out_subs:
        clear_output(wait=True)
        show = subs.head(600) if FAST_PREVIEW else subs
        figsc = px.scatter(
            show, x="sub_time_s", y="laugh_score",
            color="laugh_score", color_continuous_scale=px.colors.sequential.Plasma,
            hover_data=["mapped_text","median_lag"], height=320
        )
        figsc.update_xaxes(title_text="字幕时间（秒）")
        figsc.update_yaxes(title_text="laugh_score（窗口加权）")
        figsc.update_layout(font=dict(family=FONT_FAMILY), margin=dict(l=60,r=30,t=10,b=10))
        display(figsc)

    # 4) 表格 Top 字幕
    with out_table:
        clear_output(wait=True)
        display(subs.head(20)[["mapped_text","sub_time_s","laugh_score","median_lag"]].round(3))

    # 5) 全季进度热力图（结构节奏）
    with out_heat:
        clear_output(wait=True)
        mat, labels = heatmap_percent(df, pct_bins=HEATMAP_PCT_BINS, metric=metric, basis=basis, norm=hnorm)
        fig = go.Figure(data=go.Heatmap(z=mat, colorscale=THEME["heat_seq"],
                                        colorbar=dict(title="强度" if hnorm=="row" else "笑类总量")))
        fig.update_layout(height=340, font=dict(family=FONT_FAMILY),
                          margin=dict(l=80,r=30,t=10,b=40))
        fig.update_yaxes(tickmode="array", tickvals=list(range(len(labels))), ticktext=labels, title="集数")
        xt = np.linspace(0, HEATMAP_PCT_BINS-1, 6, dtype=int)
        fig.update_xaxes(tickmode="array", tickvals=xt,
                         ticktext=[f"{int(v*100/(HEATMAP_PCT_BINS-1))}%" for v in xt],
                         title="节目进度（%）")
        display(fig)

    # 6) 文本说明
    with out_caption:
        clear_output(wait=True)
        kws = "、".join(kw.sort_values("score", ascending=False)["term"].head(6).tolist())
        metric_str = "加权笑指数" if metric=="weighted" else "纯笑计数"
        basis_str  = "弹幕时间" if basis=="dm" else "字幕时间"
        norm_str   = "（行归一化）" if hnorm=="row" else ""
        display(HTML(f"""
        <div style="font-family:{FONT_FAMILY};font-size:13px;line-height:1.6;color:{THEME['text']}">
        <b>当前解释峰值 @ {peak_t:.1f}s</b>：窗口关键词 = {kws}<br>
        读图法：<i>时间线</i>回答“<b>何时笑</b>”（{metric_str}，高斯平滑窗={k_smooth}）；<i>关键词</i>回答“<b>为什么笑</b>”；<i>字幕散点</i>回答“<b>对什么内容笑</b>”。<br>
        右侧热力图以<b>{basis_str}</b>对齐做 0–100% 进度比较{norm_str}。
        </div>
        """))

    # 7) 内嵌你提供的两个成品 HTML（可选）
    render_html_file(HTML_HEATMAP_PATH, out_html1)
    render_html_file(HTML_WORDCLOUD_PATH, out_html2)

# 绑定事件
for w in (dd_ep, sl_bin, sl_win, sl_peak, sl_smooth, sl_prom, sl_mdist, tg_metric, tg_basis, dd_hnorm, sl_play):
    w.observe(render_all, names="value")

# ======================= 6) 页面布局与首渲染 =======================
controls_row1 = widgets.HBox([dd_ep, sl_bin, sl_win, sl_peak],
                             layout=widgets.Layout(margin="4px 0 2px 0"))
controls_row2 = widgets.HBox([sl_smooth, sl_prom, sl_mdist, tg_metric, tg_basis, dd_hnorm],
                             layout=widgets.Layout(margin="0 0 6px 0"))
controls_row3 = widgets.HBox([play, sl_play],
                             layout=widgets.Layout(margin="0 0 6px 0"))

display(HTML(f"<h3 style='font-family:{FONT_FAMILY};margin:6px 0 4px 0'>观众为何而笑 · 交互分析（Notebook 版 / 提升版）</h3>"))
display(controls_row1, controls_row2, controls_row3)
display(out_timeline)
display(widgets.HBox([widgets.VBox([out_kw]), widgets.VBox([out_subs])]))
display(out_table)
display(out_heat)
display(out_caption)
display(HTML(f"<h4 style='font-family:{FONT_FAMILY};margin:10px 0 6px 0'>补充 · 成品 HTML 预览</h4>"))
display(HTML(f"<div style='font-family:{FONT_FAMILY};color:#555;margin-bottom:6px'>如需更换路径，请修改顶部 <code>HTML_HEATMAP_PATH</code> / <code>HTML_WORDCLOUD_PATH</code>。</div>"))
display(out_html1)
display(out_html2)

render_all()  # 首次渲染